In [7]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from sklearn.preprocessing import LabelEncoder
import numpy as np
import json
import pickle

# Mô hình dialog act
dialog_act_model = load_model('/content/trained_model1.h5')

with open('/content/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('/content/metadata.json', 'r') as f:
    metadata = json.load(f)
    unique_labels = metadata['unique_labels']
    max_len = metadata['max_len']

# Mô hình BIO
bio_model = load_model('/content/model_tag.h5')

with open('/content/tokenizer.json', 'r', encoding='utf-8') as f:
    tokenizer_json = json.load(f)
tokenizer_bio = tokenizer_from_json(tokenizer_json)

with open('/content/tag_encoder.json', 'r', encoding='utf-8') as f:
    tag_encoder_classes = json.load(f)
tag_encoder = LabelEncoder()
tag_encoder.classes_ = np.array(tag_encoder_classes)

with open('/content/max_len.json', 'r', encoding='utf-8') as f:
    max_len_bio = json.load(f)['max_len']

new_sentence = "I would like to book a room for three people on Monday"

# Dự đoán dialog act
new_sentence_tokenized = tokenizer.texts_to_sequences([new_sentence])
new_sentence_padded = pad_sequences(new_sentence_tokenized, padding='post', maxlen=max_len)
dialog_act_prediction = dialog_act_model.predict(new_sentence_padded)
predicted_dialog_acts = [unique_labels[i] for i in range(len(dialog_act_prediction[0])) if dialog_act_prediction[0][i] >= 0.5]

# Dự đoán nhãn BIO
new_sentence_seq_bio = tokenizer_bio.texts_to_sequences([new_sentence])
new_sentence_seq_padded_bio = pad_sequences(new_sentence_seq_bio, padding='post', maxlen=max_len_bio)
bio_predictions = bio_model.predict(new_sentence_seq_padded_bio)
predicted_tags = np.argmax(bio_predictions, axis=-1)
predicted_bio_tags = tag_encoder.inverse_transform(predicted_tags[0])

# Kết quả dialog_act(slot: value)
slots = {}
current_slot = None
current_value = []

for word, tag in zip(new_sentence.split(), predicted_bio_tags):
    if tag.startswith("B-"):
        if current_slot:
            slots[current_slot] = " ".join(current_value)
        current_slot = tag[2:]
        current_value = [word]
    elif tag.startswith("I-") and current_slot:
        current_value.append(word)
    else:
        if current_slot:
            slots[current_slot] = " ".join(current_value)
        current_slot = None
        current_value = []

if current_slot:
    slots[current_slot] = " ".join(current_value)

# In kết quả
for act in predicted_dialog_acts:
    slot_values = ", ".join([f"{slot}: {value}" for slot, value in slots.items()])
    print(f"{act}({slot_values})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 818ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 715ms/step
Hotel-Inform(bookpeople: three, bookday: Monday)
